# Summary of ORG

This notebook contains a list of Python methods that were introduced in the ORG programming exercises.

### Import the libraries

In [1]:
import numpy as np
import pandas as pd
import sqlite3

## Step 1. Connect to database

In [2]:
conn_countries = sqlite3.connect('../datasets/countries.db')

## Step 2. Operate on database

In [3]:
query_all = 'SELECT * FROM countries;'
pd.read_sql_query(query_all, conn_countries)

,name,alpha_3,tld,continent,capital,area,population
0,Andorra,AND,.ad,EU,Andorra la Vella,468.0,84000
1,United Arab Emirates,ARE,.ae,AS,Abu Dhabi,82880.0,4975593
2,Afghanistan,AFG,.af,AS,Kabul,647500.0,29121286
3,Antigua and Barbuda,ATG,.ag,NM,St. John's,443.0,86754
4,Anguilla,AIA,.ai,NM,The Valley,102.0,13254
...,...,...,...,...,...,...,...
246,Mayotte,MYT,.yt,AF,Mamoudzou,374.0,159042
247,South Africa,ZAF,.za,AF,Pretoria,1219912.0,49000000
248,Zambia,ZMB,.zm,AF,Lusaka,752614.0,13460305
249,Zimbabwe,ZWE,.zw,AF,Harare,390580.0,13061000


### `WHERE` clause


In [4]:
query_area_population = '''
    SELECT name, capital
    FROM countries
    WHERE area < 1000 AND population > 200000;
'''
pd.read_sql_query(query_area_population, conn_countries)

,name,capital
0,Barbados,Bridgetown
1,Bahrain,Manama
2,Macao,Macao
3,Malta,Valletta
4,Maldives,Male
5,Singapore,Singapore
6,Netherlands Antilles,Willemstad


### `GROUP BY` clause


In [5]:
query = '''
    SELECT continent, COUNT(name), SUM(population), MAX(population)
    FROM countries
    GROUP BY continent
'''
pd.read_sql_query(query, conn_countries)

,continent,COUNT(name),SUM(population),MAX(population)
0,AF,58,1016879730,154000000
1,AN,5,170,140
2,AS,52,4130586341,1330044000
3,EU,53,739141025,140702000
4,NM,42,540105043,310232863
5,OC,27,36066049,21515754
6,SM,14,400143568,201103330


### `ORDER BY` clause


In [6]:
query = '''
    SELECT name, population
    FROM countries
    ORDER BY population DESC;
'''
pd.read_sql_query(query, conn_countries)

,name,population
0,China,1330044000
1,India,1173108018
2,United States,310232863
3,Indonesia,242968342
4,Brazil,201103330
...,...,...
246,South Georgia and the South Sandwich Islands,30
247,Antarctica,0
248,Bouvet Island,0
249,Heard Island and McDonald Islands,0


### `LIMIT` clause


In [7]:
query = '''
    SELECT name, population
    FROM countries
    ORDER BY population DESC
    LIMIT 10;
'''
pd.read_sql_query(query, conn_countries)

,name,population
0,China,1330044000
1,India,1173108018
2,United States,310232863
3,Indonesia,242968342
4,Brazil,201103330
5,Pakistan,184404791
6,Bangladesh,156118464
7,Nigeria,154000000
8,Russia,140702000
9,Japan,127288000


### Joining tables


In [8]:
pd.read_sql_query('SELECT * FROM continents;', conn_countries)

,code,name
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NM,North America
5,OC,Oceania
6,SM,South America


In [9]:
query = '''
    SELECT countries.name, population, continents.name AS 'continent name'
    FROM countries, continents
    WHERE countries.continent = continents.code
    ORDER BY population DESC
    LIMIT 10;
'''
pd.read_sql_query(query, conn_countries)

,name,population,continent name
0,China,1330044000,Asia
1,India,1173108018,Asia
2,United States,310232863,North America
3,Indonesia,242968342,Asia
4,Brazil,201103330,South America
5,Pakistan,184404791,Asia
6,Bangladesh,156118464,Asia
7,Nigeria,154000000,Africa
8,Russia,140702000,Europe
9,Japan,127288000,Asia


### Subqueries, or nested queries


In [10]:
query_E = '''
    SELECT continent
    FROM countries
    WHERE name LIKE 'E%'
'''
pd.read_sql_query(query_E, conn_countries)

,continent
0,SM
1,EU
2,AF
3,AF
4,AF
5,AF
6,NM
7,OC


In [11]:
query_large = '''
    SELECT continent
    FROM countries
    WHERE area > 10000000
'''
pd.read_sql_query(query_large, conn_countries)

,continent
0,AN
1,EU


In [12]:
query_E_except_large = query_E + '''
    EXCEPT
''' + query_large
print(query_E_except_large)
pd.read_sql_query(query_E_except_large, conn_countries)


    SELECT continent
    FROM countries
    WHERE name LIKE 'E%'

    EXCEPT

    SELECT continent
    FROM countries
    WHERE area > 10000000



,continent
0,AF
1,NM
2,OC
3,SM


## Step 3. Close connection to database


In [13]:
conn_countries.close()

## Data cleaning

In [14]:
country_data = pd.read_csv('../datasets/countries.csv', na_values=[''], keep_default_na=False)
country_data.head()

,name,alpha_3,tld,continent,capital,area,population
0,Andorra,AND,.ad,EU,Andorra la Vella,468.0,84000
1,United Arab Emirates,ARE,.ae,AS,Abu Dhabi,82880.0,4975593
2,Afghanistan,AFG,.af,AS,Kabul,647500.0,29121286
3,Antigua and Barbuda,ATG,.ag,NM,St. John's,443.0,86754
4,Anguilla,AIA,.ai,NM,The Valley,102.0,13254


## `df.astype()`: convert to given type

In [15]:
country_data.dtypes

name           object
alpha_3        object
tld            object
continent      object
capital        object
area          float64
population      int64
dtype: object

In [16]:
country_data_as_str = country_data.astype(str)
country_data_as_str.dtypes

name          object
alpha_3       object
tld           object
continent     object
capital       object
area          object
population    object
dtype: object

In [17]:
population_as_str = country_data.astype({'population': str})
population_as_str.dtypes

name           object
alpha_3        object
tld            object
continent      object
capital        object
area          float64
population     object
dtype: object

##  `pd.to_numeric()`: convert to numeric


In [18]:
pd.to_numeric(country_data_as_str['population']).head()

0       84000
1     4975593
2    29121286
3       86754
4       13254
Name: population, dtype: int64

In [19]:
pd.to_numeric(country_data_as_str['area']).head()

0       468.0
1     82880.0
2    647500.0
3       443.0
4       102.0
Name: area, dtype: float64

## `ts.unique()`: find all unique values


In [20]:
country_data['capital'].unique()

array(['Andorra la Vella', 'Abu Dhabi', 'Kabul', "St. John's",
       'The Valley', 'Tirana', 'Yerevan', 'Luanda', nan, 'Buenos Aires',
       'Pago Pago', 'Vienna', 'Canberra', 'Oranjestad', 'Mariehamn',
       'Baku', 'Sarajevo', 'Bridgetown', 'Dhaka', 'Brussels',
       'Ouagadougou', 'Sofia', 'Manama', 'Bujumbura', 'Porto-Novo',
       'Gustavia', 'Hamilton', 'Bandar Seri Begawan', 'Sucre', 'Brasilia',
       'Nassau', 'Thimphu', 'Gaborone', 'Minsk', 'Belmopan', 'Ottawa',
       'West Island', 'Kinshasa', 'Bangui', 'Brazzaville', 'Bern',
       'Yamoussoukro', 'Avarua', 'Santiago', 'Yaounde', 'Beijing',
       'Bogota', 'San Jose', 'Havana', 'Praia', ' Willemstad',
       'Flying Fish Cove', 'Nicosia', 'Prague', 'Berlin', 'Djibouti',
       'Copenhagen', 'Roseau', 'Santo Domingo', 'Algiers', 'Quito',
       'Tallinn', 'Cairo', 'El-Aaiun', 'Asmara', 'Madrid', 'Addis Ababa',
       'Helsinki', 'Suva', 'Stanley', 'Palikir', 'Torshavn', 'Paris',
       'Libreville', 'London', "St. Geor

## `df.duplicated()`: find duplicates


In [21]:
bm_dup_capital = country_data.duplicated(subset='capital')
country_data[bm_dup_capital]

,name,alpha_3,tld,continent,capital,area,population
29,"Bonaire, Saint Eustatius and Saba",BES,.bq,NM,NaN,328.0,18012
33,Bouvet Island,BVT,.bv,AN,NaN,49.0,0
95,Heard Island and McDonald Islands,HMD,.hm,AN,NaN,412.0,0
163,Norfolk Island,NFK,.nf,OC,Kingston,34.6,1828
220,Tokelau,TKL,.tk,OC,NaN,10.0,1466
232,United States Minor Outlying Islands,UMI,.um,OC,NaN,0.0,0


In [22]:
bm_dup_capital_all = country_data.duplicated(subset='capital', keep=False)
country_data[bm_dup_capital_all]

,name,alpha_3,tld,continent,capital,area,population
8,Antarctica,ATA,.aq,AN,NaN,14000000.0,0
29,"Bonaire, Saint Eustatius and Saba",BES,.bq,NM,NaN,328.0,18012
33,Bouvet Island,BVT,.bv,AN,NaN,49.0,0
95,Heard Island and McDonald Islands,HMD,.hm,AN,NaN,412.0,0
111,Jamaica,JAM,.jm,NM,Kingston,10991.0,2847232
163,Norfolk Island,NFK,.nf,OC,Kingston,34.6,1828
220,Tokelau,TKL,.tk,OC,NaN,10.0,1466
232,United States Minor Outlying Islands,UMI,.um,OC,NaN,0.0,0


## `df.drop_duplicates()`: remove duplicates


In [23]:
country_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        251 non-null    object 
 1   alpha_3     251 non-null    object 
 2   tld         249 non-null    object 
 3   continent   251 non-null    object 
 4   capital     245 non-null    object 
 5   area        251 non-null    float64
 6   population  251 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 13.9+ KB


In [24]:
country_data.drop_duplicates(subset='capital').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        245 non-null    object 
 1   alpha_3     245 non-null    object 
 2   tld         243 non-null    object 
 3   continent   245 non-null    object 
 4   capital     244 non-null    object 
 5   area        245 non-null    float64
 6   population  245 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 15.3+ KB


## `df.dropna()` to remove rows with missing values


In [25]:
country_data_dropna = country_data.dropna()
country_data_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 250
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        243 non-null    object 
 1   alpha_3     243 non-null    object 
 2   tld         243 non-null    object 
 3   continent   243 non-null    object 
 4   capital     243 non-null    object 
 5   area        243 non-null    float64
 6   population  243 non-null    int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 15.2+ KB


## `df.fillna()` to replace missing values


In [26]:
country_data_filled = country_data['capital'].fillna(value='Unknown')
country_data_filled[country_data_filled.duplicated(keep=False)]

8       Unknown
29      Unknown
33      Unknown
95      Unknown
111    Kingston
163    Kingston
220     Unknown
232     Unknown
Name: capital, dtype: object

In [27]:
country_data_filled = country_data.fillna({'capital': 'Unknown'})
country_data_filled

,name,alpha_3,tld,continent,capital,area,population
0,Andorra,AND,.ad,EU,Andorra la Vella,468.0,84000
1,United Arab Emirates,ARE,.ae,AS,Abu Dhabi,82880.0,4975593
2,Afghanistan,AFG,.af,AS,Kabul,647500.0,29121286
3,Antigua and Barbuda,ATG,.ag,NM,St. John's,443.0,86754
4,Anguilla,AIA,.ai,NM,The Valley,102.0,13254
...,...,...,...,...,...,...,...
246,Mayotte,MYT,.yt,AF,Mamoudzou,374.0,159042
247,South Africa,ZAF,.za,AF,Pretoria,1219912.0,49000000
248,Zambia,ZMB,.zm,AF,Lusaka,752614.0,13460305
249,Zimbabwe,ZWE,.zw,AF,Harare,390580.0,13061000


## `ts.str.strip()` to strip characters from begin/end of strings


In [28]:
country_data_as_str['area'].str.strip('0')

0          468.
1        82880.
2       647500.
3          443.
4          102.
         ...   
246        374.
247    1219912.
248     752614.
249     390580.
250        960.
Name: area, Length: 251, dtype: object